# Projeto Final - Análise de Gols em uma Partida de Futebol
Autor: Paulo Victor Lima |   Orientador : Sergio Lima Netto |    Universidade Federal do Rio de Janeiro - Escola Politécnica - Departamente de Engenharia Eletrônica e de Computação

## Importação das Bibliotecas

In [ ]:
#Instalação das bibliotecas

!pip install pandas
!pip install numpy
!pip install seaborn
!pip install matplotlib.pyplot
!pip install plotly.express
!pip install glob

In [5]:
#Importação das bibliotecas

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import glob

## Importação Base de Dados